### 导入包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier,XGBRegressor
from mlxtend.classifier import StackingClassifier
import lightgbm as lgb
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import confusion_matrix,accuracy_score, recall_score, f1_score, precision_score,confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict


import warnings
warnings.filterwarnings("ignore")


### 导入数据

In [2]:
data=pd.read_csv('./clean-data.csv')
data=data.drop(['Unnamed: 0'],axis=1)
X=data.drop(['Group'],axis=1)
y=data['Group']

### 数据归一化

In [3]:
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)

### 划分训练集测试集

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
print(f'Train shape : {X_train.shape}\nTest shape: {X_test.shape}')

Train shape : (132, 203)
Test shape: (33, 203)


### 模型

In [5]:
#只在测试集上测试
def testpredict(estimator,Xtrain,ytrain,Xtest,ytest):
    print("只在测试集上测试")
    model=estimator.fit(Xtrain,ytrain)
    y_pred_train=model.predict(Xtrain)
    y_pred_test=model.predict(Xtest)
    confusion=confusion_matrix(ytest,y_pred_test)
    acc=accuracy_score(ytest, y_pred_test)
    rec=recall_score(ytest, y_pred_test)
    pre=precision_score(ytest, y_pred_test)
    f1=f1_score(ytest, y_pred_test)
    print('the confusion_matrix of the model is : \n',confusion)
    print('the accuracy of the model on testing set is ：',acc)
    print("the precision score of the model on testing set is : ",pre )
    print("the recall score of the model on testing set is :", rec)
    print('the f1_score of the model on testing set is :',f1)
    
    return acc,pre,rec,f1

In [6]:
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("cross-validate across the entire data set")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    a=accuracy_score(ytrain,y_pred_cross)
    p=precision_score(ytrain, y_pred_cross)
    r=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    wf1=f1_score(ytrain, y_pred_cross, average='weighted')
    #auc=roc_auc_score(ytrain,estimator.predict_proba(Xtrain)[:,1])
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',a)
    print("the precision score of the model is : ", p)
    print("the recall score of the model is :", r)
    print('the f1_score of the model  is :',f1)
    print('the weighted_f1 of the model is :',wf1)
    print('the classification_report is :\n',classification_report(ytrain, y_pred_cross,digits=4))
    #print('the auc is :',auc)
    return a,p,r,f1,wf1

#### SVC(),RandomForestClassifier(),KNeighborsClassifier(),AdaBoostClassifier(),GaussianNB(),BernoulliNB(),
   #### LogisticRegression(),GradientBoostingClassifier(),XGBClassifier(),lgb.LGBMClassifier()

### 网格搜索

In [7]:
#Grid Search
import sklearn.model_selection as ms
def gridsearch(params,estimator,Xtrain,ytrain,cvnumber):
    model = ms.GridSearchCV(estimator, params, cv=cvnumber)
    model.fit(Xtrain, ytrain)
    print("the best_params of the model is:",model.best_params_)
    print("the best_score of the model is:",model.best_score_)
    print("the best_estimator of the model is:",model.best_estimator_)

In [12]:
## svm
params = [
    {'kernel':['linear'], 'C':[1, 10, 100, 1000]},
    {'kernel':['poly'], 'C':[1,10,100], 'degree':[2, 3,4,5],'coef0':[0,1,10,100]}, 
    {'kernel':['rbf'], 'C':[1,10,100], 'gamma':[1, 0.1, 0.01]}]
gridsearch(params,SVC(probability=True),X_train,y_train,10)

the best_params of the model is: {'C': 1, 'coef0': 10, 'degree': 4, 'kernel': 'poly'}
the best_score of the model is: 0.7714285714285716
the best_estimator of the model is: SVC(C=1, coef0=10, degree=4, kernel='poly', probability=True)


In [26]:
#RF
params={
    'n_estimators':np.arange(1,201,1),
#     'max_depth': np.arange(1,20,1),
}
model = ms.GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=90), params, cv=10)
model.fit(X_train, y_train)

print("the best_params of the RF model is:",model.best_params_)
# 获取最优得分
print("the best_score of the RF model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the RF model is:",model.best_estimator_)

the best_params of the RF model is: {'n_estimators': 105}
the best_score of the RF model is: 0.7186813186813186
the best_estimator of the RF model is: RandomForestClassifier(n_estimators=105, n_jobs=-1, random_state=90)


In [27]:
#RF
params={
#     'n_estimators':np.arange(1,201,1),
     'max_depth': np.arange(1,20,1),
}
model = ms.GridSearchCV(RandomForestClassifier(n_estimators=105, n_jobs=-1,random_state=90), params, cv=10)
model.fit(X_train, y_train)

print("the best_params of the RF model is:",model.best_params_)
# 获取最优得分
print("the best_score of the RF model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the RF model is:",model.best_estimator_)

the best_params of the RF model is: {'max_depth': 8}
the best_score of the RF model is: 0.7252747252747251
the best_estimator of the RF model is: RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,
                       random_state=90)


In [9]:
#KNN
params={
    'n_neighbors': [3, 5, 7, 9],  # 邻居的数量
    'weights': ['uniform', 'distance'],  # 权重函数
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # 算法类型
    'leaf_size': [30, 40, 50],  # 叶节点的大小
    'p': [1, 2]  # 距离度量的幂次
}
gridsearch(params,KNeighborsClassifier(),X_train, y_train,10)

the best_params of the model is: {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
the best_score of the model is: 0.689010989010989
the best_estimator of the model is: KNeighborsClassifier(n_neighbors=3, p=1)


In [29]:
#AdaBoost
params={
    'n_estimators':[10,50,100,200,300,400,500],
    'learning_rate':[0.0001,0.001,0.01,0.1,1]
}
model = ms.GridSearchCV(AdaBoostClassifier(), params, cv=10)
model.fit(X_train, y_train)

print("the best_params of the AdaBoost model is:",model.best_params_)
# 获取最优得分
print("the best_score of the AdaBoost model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the AdaBoost model is:",model.best_estimator_)

the best_params of the AdaBoost model is: {'learning_rate': 0.1, 'n_estimators': 50}
the best_score of the AdaBoost model is: 0.7725274725274726
the best_estimator of the AdaBoost model is: AdaBoostClassifier(learning_rate=0.1)


In [30]:
#LR
params={
    'C':[0.0001,0.001,0.01,0.1],
    'max_iter':[100,200,300,400,500]
}
model = ms.GridSearchCV(LogisticRegression(), params, cv=10)
model.fit(X_train, y_train)
print("the best_params of the LogisticRegression model is:",model.best_params_)
# 获取最优得分
print("the best_score of the LogisticRegression model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the LogisticRegression model is:",model.best_estimator_)

the best_params of the LogisticRegression model is: {'C': 0.1, 'max_iter': 100}
the best_score of the LogisticRegression model is: 0.787912087912088
the best_estimator of the LogisticRegression model is: LogisticRegression(C=0.1)


In [31]:
#Gradient Boosting
params={
    'loss':['deviance','exponential'],
    'n_estimators':[10,50,100,200,300,400,500],
    'learning_rate':[0.0001,0.001,0.01,0.1,1]
}
model = ms.GridSearchCV(GradientBoostingClassifier(), params, cv=10)
model.fit(X_train, y_train)
print("the best_params of the GradientBoost model is:",model.best_params_)
# 获取最优得分
print("the best_score of the GradientBoost model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the GradientBoost model is:",model.best_estimator_)

the best_params of the GradientBoost model is: {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 300}
the best_score of the GradientBoost model is: 0.7879120879120879
the best_estimator of the GradientBoost model is: GradientBoostingClassifier(loss='exponential', n_estimators=300)


In [11]:
#XGBoost
params={
    'booster':['gbtree','gblinear'],
    'n_estimators':[10,50,100,200,300,400,500,1000,2000],
    'learning_rate':[0.0001,0.001,0.01,0.05,0.1,0.3,1],
    'max_depth': [5, 10, 15, 20, 25]
}
model = ms.GridSearchCV(XGBClassifier(), params, cv=10)
model.fit(X_train, y_train)
print("the best_params of the XGBoost model is:",model.best_params_)
# 获取最优得分
print("the best_score of the XGBoost model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the XGBoost model is:",model.best_estimator_)

[12:26:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:26:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:26:53] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:53] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:53] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:26:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:56] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:56] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:27:05] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:07] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

...

[12:37:35] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:35] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:35] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:37:36] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:36] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:36] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:37:38] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:38] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:38] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:37:41] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:41] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:41] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

[12:37:47] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:37:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depth" } might not be used.

  This could b

In [ ]:
# 'booster': 'gbtree', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 1   0.74

In [12]:
#LightGBM
params={
    'n_estimators': [100,200,300,400,500],
    'learning_rate': [0.01,0.015,0.025,0.05,0.1],
    'max_depth':[3,5,6,7,9,12,17,25]
}
gridsearch(params,lgb.LGBMClassifier(),X_train,y_train,10)

the best_params of the model is: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
the best_score of the model is: 0.6967032967032967
the best_estimator of the model is: LGBMClassifier(max_depth=3, n_estimators=300)


In [1]:
#decisiontree
from sklearn.tree import DecisionTreeClassifier
params={
    'max_depth':list((5,10,12)),
    'min_samples_split': list((2,4,6)),
    'min_samples_leaf':list((1,2,4))
}
gridsearch(params,DecisionTreeClassifier(),X_train,y_train,10)

the best_params of the model is: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 6}
the best_score of the model is: 0.7131868131868132
the best_estimator of the model is: DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6)

In [2]:
import sklearn.model_selection as ms
#GausianNB
nb = GaussianNB()
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}
# 使用GridSearchCV进行网格搜索
model=ms.GridSearchCV(nb, param_grid=param_grid, cv=10)
model.fit(X_train, y_train)
print("the best_params of the GaussianNB model is:",model.best_params_)
# 获取最优得分
print("the best_score of the GaussianNB model is:",model.best_score_)
# 获取最优模型的信息
print("the best_estimator of the GaussianNB model is:",model.best_estimator_)

the best_params of the GaussianNB model is: {'var_smoothing': 1e-09}
the best_score of the GaussianNB model is: 0.673076923076923
the best_estimator of the GaussianNB model is: GaussianNB()